In [58]:
import json

In [40]:
data = sqlContext.read.json("reviewData/005_Helsinki_tripadvisor.json")

In [41]:
type(data)

pyspark.sql.dataframe.DataFrame

In [42]:
data.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- date: string (nullable = true)
 |-- hotelLocation: string (nullable = true)
 |-- hotelName: string (nullable = true)
 |-- hotelStars: string (nullable = true)
 |-- hotelUrl: string (nullable = true)
 |-- review: string (nullable = true)
 |-- score: double (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- userId: string (nullable = true)



In [43]:
from pyspark.sql import Row

data dictionary demo:

{"city":"Helsinki",
"website": "tripadvisor",
"totalReviews": 23410,
"totalHotels": xxx
"eachHotelDetail":
    [{"hotelUrl": xxx
    "reviewsNum": xxx
    "5stars": xxx,
    "4stars": xxx,
    "3stars": xxx,
    "2stars": xxx,
    "1stars": xxx,
    "avgScore": xxx}
    {}
    ]
"5 score": xxx,
"4 score": xxx,
"3 score": xxx,
"2 score": xxx,
"1 score": xxx,
"avgScore": xxx,
"maxReviewNumAHotel": xxx,
"minReviewNumAHotel": xxx,
"avgReviewNumAHotel": xxx,
"maxWordInAReview": xxx,
"minWordInAReview": xxx,
"AvgWordInAReview": xxx,
}

In [44]:
statistics = dict()

In [45]:
#count how many review record for the city hotels
data.count()

23410

count the number of unique reviews for the city hotels

In [46]:

statistics["totalReviews"] = data.select("url").distinct().count()

calculate the number of unique hotels and each of hotel get how many reviews

In [47]:
# try to remove duplicate reviews
data = data.drop_duplicates()

In [48]:
hotelsList = sorted(data.groupBy(data.hotelUrl).count().collect())

In [49]:
type(hotelsList[0])

pyspark.sql.types.Row

In [50]:


statistics["maxReviewNumAHotel"] = 0
statistics["minReviewNumAHotel"] = statistics["totalReviews"]

In [56]:
statistics["eachHotelDetail"] = list()
statistics["totalHotels"] = len(hotelsList)
statistics["avgReviewNumAHotel"] = statistics["totalReviews"] / len(hotelsList)
totalScore = 0
totalScoreForOneHotel = 0
#store the number which don't give score for that review
nullScoreReviews = 0
#calculate statistics for each hotel in this city
for row in hotelsList:
    hotelDetails = dict()
    oneHotelData = data.filter(data.hotelUrl == row["hotelUrl"])
    hotelDetails["hotelUrl"] = row["hotelUrl"]
    
    try:
#         hotelStars means hotel class for this hotel, not the review score.
        hotelDetails["hotelStars"] = data.filter(data.hotelUrl == row["hotelUrl"]).first()["hotelStars"]
    except:
        hotelDetails["hotelStars"] = None
    
    
    
    hotelDetails["reviewsNum"] = oneHotelData.count()
    
    if hotelDetails["reviewsNum"] > statistics["maxReviewNumAHotel"]:
        statistics["maxReviewNumAHotel"] = hotelDetails["reviewsNum"]
    if hotelDetails["reviewsNum"] < statistics["minReviewNumAHotel"]:
        statistics["minReviewNumAHotel"] = hotelDetails["reviewsNum"]
    
    nullScoreReviewsAHotel = 0
    hotelDetails["avgScore"] = 0
    for scoreRow in oneHotelData.groupby(oneHotelData.score).count().collect():
#         if the user doesn't give any score, then 
        if scoreRow["score"] == None:
            k = "null score"
            nullScoreReviews += 1
            nullScoreReviewsAHotel += 1
        else:
            k = str(scoreRow["score"]) + " score"
            totalScoreForOneHotel += float(scoreRow["score"]) * float(scoreRow["count"])
        hotelDetails[k] = scoreRow["count"]
    #add total score that each hotel gets
    totalScore += totalScoreForOneHotel
    hotelDetails["avgScore"] = totalScoreForOneHotel / (hotelDetails["reviewsNum"] - nullScoreReviewsAHotel)
    totalScoreForOneHotel = 0
    statistics["eachHotelDetail"].append(hotelDetails)

statistics["avgScore"] = totalScore / (statistics["totalReviews"] - nullScoreReviews)

In [52]:
#statistics

In [53]:
#classify all reviews regardless hotels into 6 categories, 0, 1, 2, 3, 4, 5 scores 
for scoreCityRow in data.groupBy(data.score).count().collect():
    if scoreCityRow["score"] == None:
        kCity = "null score"
    else:
        kCity = str(scoreCityRow["score"]) + " score"
    statistics[kCity] = scoreCityRow["count"]

In [54]:
oneHotelData = data.filter(data.hotelUrl == hotelsList[22]["hotelUrl"])
oneHotelData.groupby(oneHotelData.score).count().collect()

[Row(score=None, count=8),
 Row(score=1.0, count=21),
 Row(score=4.0, count=373),
 Row(score=3.0, count=187),
 Row(score=2.0, count=62),
 Row(score=5.0, count=144)]

In [57]:
statistics

{'1.0 score': 342,
 '2.0 score': 929,
 '3.0 score': 3532,
 '4.0 score': 9461,
 '5.0 score': 9007,
 'avgReviewNumAHotel': 450.1923076923077,
 'avgScore': 4.089549048942081,
 'eachHotelDetail': [{'1.0 score': 2,
   '3.0 score': 1,
   '4.0 score': 27,
   '5.0 score': 270,
   'avgScore': 4.876666666666667,
   'hotelStars': '4.0',
   'hotelUrl': 'https://www.tripadvisor.com/Hotel_Review-g189934-d10263887-Reviews-Hotel_F6-Helsinki_Uusimaa.html',
   'reviewsNum': 300},
  {'1.0 score': 2,
   '2.0 score': 3,
   '3.0 score': 20,
   '4.0 score': 28,
   '5.0 score': 24,
   'avgScore': 3.896103896103896,
   'hotelStars': '4.0',
   'hotelUrl': 'https://www.tripadvisor.com/Hotel_Review-g189934-d1027051-Reviews-Radisson_Blu_Aleksanteri-Helsinki_Uusimaa.html',
   'reviewsNum': 77},
  {'1.0 score': 1,
   '5.0 score': 1,
   'avgScore': 3.0,
   'hotelStars': '3.0',
   'hotelUrl': 'https://www.tripadvisor.com/Hotel_Review-g189934-d1146298-Reviews-Kulttuurikeskus_Sofia-Helsinki_Uusimaa.html',
   'reviewsNum

In [60]:
with open("005_Statistics_Helsinki.json", "w") as f:
    json.dump(statistics, f)